In [1]:
import pandas as pd
from pathlib import Path
import re

In [2]:
# === Caminhos ===
DADOS_DIR = Path("Dados/")
RESULTADOS_DIR = Path("Resultados/")
RESULTADOS_DIR.mkdir(exist_ok=True)

In [3]:
# === Função de limpeza ===
def limpar_titulo(title):
    if not isinstance(title, str):
        return ""
    title = title.strip().lower()
    # Remove caracteres especiais e espaços extras
    title = re.sub(r"[^a-z0-9\s]", "", title)
    return title.strip()

In [4]:
# === Função principal ===
def extrair_catalogo_netflix():
    print("🎬 Carregando dataset do Kaggle - Catálogo da Netflix...\n")

    # 1️⃣ Carregar o dataset do Kaggle (arquivo: netflix_titles.csv)
    df = pd.read_csv(DADOS_DIR / "netflix_titles.csv")
    print(f"✅ {len(df):,} títulos carregados com {len(df.columns)} colunas originais.\n")

    # 2️⃣ Selecionar e renomear colunas principais
    df = df.rename(columns={
        "title": "titulo",
        "type": "tipo",
        "release_year": "ano_lancamento",
        "listed_in": "generos",
        "duration": "duracao"
    })[["titulo", "tipo", "ano_lancamento", "generos", "duracao", "country", "rating", "date_added"]]

    # 3️⃣ Limpeza e padronização
    df["titulo_limpo"] = df["titulo"].apply(limpar_titulo)
    df["ano_lancamento"] = pd.to_numeric(df["ano_lancamento"], errors="coerce").fillna(-1).astype(int)
    df["duracao_min"] = (
        df["duracao"]
        .fillna("0")
        .apply(lambda x: int(re.findall(r"\d+", x)[0]) if re.findall(r"\d+", x) else -1)
    )

    # 4️⃣ Criar variáveis auxiliares
    df["tipo"] = df["tipo"].str.lower()
    df["pais_disponivel"] = df["country"].fillna("Desconhecido")
    df["generos"] = df["generos"].fillna("Desconhecido")

    # 5️⃣ Remover duplicatas
    df = df.drop_duplicates(subset=["titulo_limpo", "ano_lancamento"]).reset_index(drop=True)

    # 6️⃣ Salvar catálogo limpo
    df.to_parquet(RESULTADOS_DIR / "Catalogo_Netflix_Limpo.parquet", index=False, engine="fastparquet")

    print("💾 Catálogo limpo salvo em 'Resultados/Catalogo_Netflix_Limpo.parquet'")
    print(f"📊 Total de títulos únicos: {len(df):,}")

    return df

# Executar
catalogo_df = extrair_catalogo_netflix()
catalogo_df.head(10)

🎬 Carregando dataset do Kaggle - Catálogo da Netflix...

✅ 8,807 títulos carregados com 12 colunas originais.

💾 Catálogo limpo salvo em 'Resultados/Catalogo_Netflix_Limpo.parquet'
📊 Total de títulos únicos: 8,799


,titulo,tipo,ano_lancamento,generos,duracao,country,rating,date_added,titulo_limpo,duracao_min,pais_disponivel
0,Dick Johnson Is Dead,movie,2020,Documentaries,90 min,United States,PG-13,"September 25, 2021",dick johnson is dead,90,United States
1,Blood & Water,tv show,2021,"International TV Shows, TV Dramas, TV Mysteries",2 Seasons,South Africa,TV-MA,"September 24, 2021",blood water,2,South Africa
2,Ganglands,tv show,2021,"Crime TV Shows, International TV Shows, TV Act...",1 Season,NaN,TV-MA,"September 24, 2021",ganglands,1,Desconhecido
3,Jailbirds New Orleans,tv show,2021,"Docuseries, Reality TV",1 Season,NaN,TV-MA,"September 24, 2021",jailbirds new orleans,1,Desconhecido
4,Kota Factory,tv show,2021,"International TV Shows, Romantic TV Shows, TV ...",2 Seasons,India,TV-MA,"September 24, 2021",kota factory,2,India
5,Midnight Mass,tv show,2021,"TV Dramas, TV Horror, TV Mysteries",1 Season,NaN,TV-MA,"September 24, 2021",midnight mass,1,Desconhecido
6,My Little Pony: A New Generation,movie,2021,Children & Family Movies,91 min,NaN,PG,"September 24, 2021",my little pony a new generation,91,Desconhecido
7,Sankofa,movie,1993,"Dramas, Independent Movies, International Movies",125 min,"United States, Ghana, Burkina Faso, United Kin...",TV-MA,"September 24, 2021",sankofa,125,"United States, Ghana, Burkina Faso, United Kin..."
8,The Great British Baking Show,tv show,2021,"British TV Shows, Reality TV",9 Seasons,United Kingdom,TV-14,"September 24, 2021",the great british baking show,9,United Kingdom
9,The Starling,movie,2021,"Comedies, Dramas",104 min,United States,PG-13,"September 24, 2021",the starling,104,United States
